# Assignment 10.2

> Replace all TODOs with your code. Do not change any other code.

In [ ]:
# Do not edit this cell

import csv
import unittest


## Clean code

### Task 1

You are given a function that reads a csv file with temperature measurements (see example below), converts Fahrenheit values to Celsius, calculates and prints some statistics, and writes to another file. It looks a bit messy, let's clean it up!

Example file:
```csv
Temperature (F)
78.5
81.2
75.9
82.1
```

Do the steps below one by one, editing the code in the cell:
1. Naming is so ambiguous and unclear, let's rename variables and function name with proper names.
2. Are these comments really useful?
3. This function does quite a lot, let's divide it in the way that each function does only one thing, and there's one main function that uses others.
4. There seem to be some magic coefficients in the temperature conversion part; let's make them obvious.

If you find any additional improvements, feel free to implement them and leave a comment under your code with an explanation.

In [14]:
def read_fahrenheit_temperatures(input_file):
    temperatures_fahrenheit = []
    with open(input_file, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            fahrenheit = float(row[0])
            temperatures_fahrenheit.append(fahrenheit)
    return temperatures_fahrenheit


def convert_fahrenheit_to_celsius(fahrenheit_temp):
    FAHRENHEIT_TO_CELSIUS_FACTOR = 5 / 9
    FAHRENHEIT_FREEZING_POINT = 32
    return (fahrenheit_temp - FAHRENHEIT_FREEZING_POINT) * FAHRENHEIT_TO_CELSIUS_FACTOR


def calculate_statistics(temperatures_celsius):
    average = sum(temperatures_celsius) / len(temperatures_celsius)
    minimum = min(temperatures_celsius)
    maximum = max(temperatures_celsius)
    return average, minimum, maximum


def write_celsius_temperatures(output_file, temperatures_celsius):
    with open(output_file, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Temperature (C)'])
        for temp in temperatures_celsius:
            writer.writerow([temp])


def process_temperature_data(input_file, output_file):
    fahrenheit_temps = read_fahrenheit_temperatures(input_file)
    celsius_temps = [convert_fahrenheit_to_celsius(f) for f in fahrenheit_temps]
    average, minimum, maximum = calculate_statistics(celsius_temps)

    print("Statistics:")
    print("Average: {:.2f}°C".format(average))
    print("Minimum: {:.2f}°C".format(minimum))
    print("Maximum: {:.2f}°C".format(maximum))

    write_celsius_temperatures(output_file, celsius_temps)

### Task 2

How would you write tests for the initial implementation? What exactly would you test in the function?

I hope you see now that once functionality is separated, it's easier to test it in isolation. So, let's write a couple of unit tests for your function and one integration test for your main function.

Hint: you would probably want to mock reading from/writing to file to make the test independent from the environment.

In [13]:
from unittest.mock import mock_open, patch
import io


class UnitTestCase(unittest.TestCase):

    def test_convert_fahrenheit_to_celsius(self):
        self.assertAlmostEqual(convert_fahrenheit_to_celsius(32), 0)
        self.assertAlmostEqual(convert_fahrenheit_to_celsius(212), 100)
        self.assertAlmostEqual(convert_fahrenheit_to_celsius(98.6), 37, places=1)

    def test_calculate_statistics(self):
        temps = [0, 10, 20]
        average, minimum, maximum = calculate_statistics(temps)
        self.assertEqual(average, 10)
        self.assertEqual(minimum, 0)
        self.assertEqual(maximum, 20)

    def test_read_fahrenheit_temperatures(self):
        mock_csv = "Temperature (F)\n32\n212\n"
        with patch("builtins.open", mock_open(read_data=mock_csv)):
            result = read_fahrenheit_temperatures("dummy.csv")
            self.assertEqual(result, [32.0, 212.0])

    def test_write_celsius_temperatures(self):
        temps = [0, 100]
        mock_output = io.StringIO()
        with patch("builtins.open", return_value=mock_output, create=True):
            write_celsius_temperatures("dummy.csv", temps)
            mock_output.seek(0)
            content = mock_output.read()
            self.assertIn("Temperature (C)", content)
            self.assertIn("0", content)
            self.assertIn("100", content)


class IntegrationTestCase(unittest.TestCase):

    @patch("builtins.open", new_callable=mock_open, read_data="Temperature (F)\n32\n212\n")
    def test_process_temperature_data(self, mock_file):
        output_file = io.StringIO()
        with patch("csv.writer") as mock_writer_class:
            mock_writer = mock_writer_class.return_value
            mock_writer.writerow = lambda row: None

            process_temperature_data("input.csv", "output.csv")

            self.assertTrue(mock_writer_class.called)
            self.assertTrue(mock_writer.writerow.called)